In [1]:
from langchain_groq import ChatGroq

In [12]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)

In [20]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/director-quality-engineering/job/R-57829")
page_data = loader.load().pop().page_content
print(page_data)






















Director, Quality Engineering










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu

In [21]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        Do not include any explanation, comments, or preamble.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Director, Quality Engineering',
  'experience': '5 years',
  'skills': ['JAVA',
   'Python',
   'JSON',
   'API',
   'XML',
   'SQL Developer',
   'IBM Sterling',
   'SAP-AFS',
   'SAP S4',
   'EC2',
   'Dynamo DB',
   'SQS',
   'SNS',
   'Jenkins',
   'POSTMAN',
   'JIRA',
   'Splunk',
   'Advanced REST Client',
   'Manhattan WMS',
   'Supply chain Distribution Center and Logistics'],
  'description': 'Build and lead the quality engineering team across the company’s technology landscape on marketplace supply chain and product supply chain strategies; Strategize and plan three to five year roadmap to build out architecture while testing the plan to execute the validation; Analyze, understand, and review the business and technical requirements of the company’s objectives; Build, review and update automation test scripts to ensure automated testing is successful; Perform gap analysis of company requirements and architecture; Review test executions performed by different qualit

In [23]:
type(json_res)

list

In [24]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [31]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [33]:
all_links = [
    collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
    for job in json_res
]

In [34]:
print(all_links)

[[[{'links': 'https://example.com/java-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/flutter-portfolio'}], [{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links': 'https://example.com/angular-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/java-portfolio'}], [{'links': 'https://example.com/ios-ar-portfolio'}, {'links': 'https://example.com/angular-portfolio'}], [{'links': 'https://example.com/java-portfolio'}, {'links': 'https://example.com/kotlin-backend-portfolio'}], [{'links': 'https://example.com/

In [35]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Quality Engineering Solutions for Your Supply Chain Needs

Dear [Hiring Manager's Name],

I came across your job posting for a Director, Quality Engineering, and I'm excited to introduce AtliQ, an AI & Software Consulting company dedicated to facilitating seamless business process integration through automation. Our expertise in quality engineering can help you build and lead a high-performing team across your technology landscape.

With over 5 years of experience in quality engineering, we've developed a strong proficiency in JAVA, Python, JSON, API, XML, SQL Developer, IBM Sterling, SAP-AFS, SAP S4, EC2, Dynamo DB, SQS, SNS, Jenkins, POSTMAN, JIRA, Splunk, Advanced REST Client, Manhattan WMS, and Supply chain Distribution Center and Logistics.

Our team has a proven track record of performing gap analysis, reviewing technical architecture, and managing issues across cross-functional teams. We've successfully strategized and planned three-to-five-year roadmaps to build